In [40]:
import selenium 
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
from selenium.webdriver import Chrome

In [11]:
browser=Chrome()

In [12]:
url='https://appsrv.pace.edu/ScheduleExplorerLive/'
browser.get(url)

In [13]:
clear_link=browser.find_element_by_link_text('CLEAR ABOVE FORM')

In [14]:
clear_link.click()

In [15]:
from selenium.webdriver.support.ui import Select

In [16]:
term_select=Select(browser.find_element_by_id('checkterm'))
term_select.select_by_visible_text('Fall 2020')

In [17]:
level_select=Select(browser.find_element_by_id('level'))

In [18]:
level_select.select_by_value('Graduate')

In [19]:
subject_select=Select(browser.find_element_by_id('subject'))

In [20]:
subject_options=subject_select.options

In [21]:
is_options=[subject.text for subject in subject_options if '(IS)' in subject.text]

In [22]:
is_option=is_options[0]

is_option

'Information Systems (IS)'

In [23]:
subject_select.select_by_visible_text(is_option)

In [24]:
browser.find_element_by_id('submitbutton').click()

In [25]:
### Part 2 

In [26]:
html=browser.page_source

In [30]:
tables=pd.read_html(html)

In [36]:
results=tables[0]

In [61]:
crns=results['CRN'].tolist()

In [62]:
crns

[73592,
 70426,
 70376,
 71721,
 72779,
 70063,
 71371,
 70377,
 71427,
 72669,
 70222,
 70261,
 71363,
 71970,
 72218,
 72177,
 72023,
 72174,
 72467,
 73593,
 72044,
 72175,
 71749,
 72043,
 70736,
 72455,
 72599,
 72469,
 71519,
 74062,
 73733,
 71893,
 73594]

In [63]:
### Alternative Way 

soup=BeautifulSoup(html, 'html.parser')
all_links=soup.find_all('a' , {'data-toggle':'modal'})

In [64]:
all_links[0].text.strip().isnumeric()

True

In [65]:
crns2=[link.text.strip() for link in all_links if link.text.strip().isnumeric()==True]

In [66]:
import time



In [67]:
#browser.refresh()
a_crn

'73592'

In [68]:
pd.read_html(popup.get_attribute('innerHTML'))[0].T


,0,1,2,3,4,5,6,7,8,9
0,COURSE TITLE:,SUBJECT/COURSE#:,CRN:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
1,Info Security Auditing & Risk,IS 643,72467,3.000,Seidenberg School of CSIS,Information Systems,Graduate,None,IS 641 Minimum Grade of C,Course Description: This course provides an in...


In [71]:
all_dfs=[]
for a_crn in crns2:
    print(a_crn)
    crn_s_link=browser.find_element_by_link_text(a_crn)
    browser.execute_script("return arguments[0].scrollIntoView();", crn_s_link)
    time.sleep(.5)
    crn_s_link.click()
    time.sleep(.5)
    popup=browser.find_element_by_id('modalBodyContentDiv')

    popup_html=popup.get_attribute('innerHTML')

    df=pd.read_html(popup_html)[0].T

    cols=df.iloc[0]

    df=df[1:]

    df.columns=cols
    
    all_dfs.append(df)



    close_button=browser.find_element_by_tag_name('Button')
    time.sleep(.5)
    close_button.click()

73592
70426
70376
71721
72779
70063
71371
70377
71427
72669
70222
70261
71363
71970
72218
72177
72023


ElementNotInteractableException: Message: element not interactable: element has zero size
  (Session info: chrome=86.0.4240.80)


In [260]:
all_dfs=pd.concat(all_dfs)

In [265]:
all_dfs=all_dfs.rename({'CRN:':'CRN'}, axis='columns')


In [271]:
all_dfs['CRN']=all_dfs['CRN'].astype(int)

In [273]:
final_results=results.merge(all_dfs, how='left', on='CRN')

In [274]:
final_results

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,...,More Info,COURSE TITLE:,SUBJECT/COURSE#:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Advanced Spreadsheet Skills,IS 522,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,70426,IS,612,Introduction to Coding,LEC,3,NYC,Web-assisted (WA),NaN,W,...,More Info,Introduction to Coding,IS 612,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
2,70376,IS,613,Database Management Systems,LEC,3,NYC,Web-assisted (WA),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
3,71721,IS,613,Database Management Systems,LEC,3,Online,Synchronous (SYNC),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
4,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
5,70063,IS,617,Information Systems Principles,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Information Systems Principles,IS 617,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course examines manag...
6,71371,IS,620,IS and Organizational Strategy,LEC,3,NYC,HyFlex (HYFLX),NaN,W,...,More Info,Information Systems and Organizational Strategy,IS 620,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course brings to life...
7,70377,IS,623,Info Sys Design & Devlpmt,LEC,3,NYC,HyFlex (HYFLX),NaN,M,...,More Info,Information Systems Design and Development,IS 623,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Not open to students who have taken IS 621. Co...
8,71427,IS,623,Info Sytems Design & Devlpmt,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Information Systems Design and Development,IS 623,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Not open to students who have taken IS 621. Co...
9,72669,IS,628,Research Mthds for User Exp.,LEC,3,Online,Mixed (MIXAS),"Meeting 163 William St, Rm 238",W,...,More Info,Research Methods for User Experience,IS 628,3.000,Seidenberg School of CSIS,Information Systems,Graduate,None,None,"Course Description: In User Experience, the fo..."
